# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [1]:
import json
try:
	# This case works in ros1 environments
	from knowrob.kb import *
except ImportError:
	# If the import fails, import the knowrob.so directly
	from knowrob import *
    
InitKnowRob()

[12:39:34.614] [info] [KnowRob] static initialization done.


### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [3]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"}
		]
	},
	"data-sources": [
		{"path": "owl/test/USD.owl", "format": "rdf-xml"},
		{"path": "owl/test/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],
	"reasoner": []
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

[12:39:53.730] [info] Using backend `redland` with type `Redland`.
[12:39:53.739] [info] [redland] using storage of type "memory" with name "knowrob" and options "contexts='yes'".
[12:39:53.739] [info] Using queryable backend with id 'redland'.
[12:39:53.740] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/rdf-schema.xml' with version "Mon Apr 22 16:37:36 2024" and origin "rdf-schema".
[12:39:53.743] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/owl.rdf' with version "Mon Apr 22 16:37:36 2024" and origin "owl".
[12:39:53.750] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/test/USD.owl' with version "Wed Aug 14 13:42:10 2024" and origin "USD".
[12:39:53.778] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/external/DUL.owl' with version "27-08-2024" and origin "DUL".
[12:39:53.823] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/test/house_4.owl' with version "Tue Aug 20 12:42:54 2024" 

[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#matrix4d treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#token_array treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#matrix4d treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#token_array treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#matrix4d treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#token_array treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#matrix4d treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#token_array treated as string.
[12:39:53.851] [warning] Unknown XSD type IRI https://ease-crc.org/ont/USD.owl#matrix4d treated as string.
[12:39:53.852] [warning] 

### Submitting a Query

Now we want to run a query on the loaded semantic map:

In [4]:
phi1 = QueryParser.parse("triple('https://ease-crc.org/ont/USD.owl#house_4', 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#hasPart', ?y)")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))

[12:40:02.068] [error] [redland] syntax error, unexpected '.'
[12:40:02.068] [warning] Failed to execute query `PREFIX DUL: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
SELECT ?y 
WHERE {
    DUL:hasPart ?y . 
}` for model "".


CQ1 Which objects do I need for breakfast/
CQ2 Which objects contain something to drink?
CQ3 Where do we expect an item to be?
CQ4 What can I grasp on an object to open it?
CQ5 Where should I put the utensils for breakfast?

In [5]:
query = "rdfs:subClassOf(Beverage, 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#hasPart')"
phi1 = QueryParser.parse(query)
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))